In [9]:
import requests; from urllib.parse import urlparse
from urllib.request import urlopen
import pandas as pd

import re

In [52]:
#데이터를 정상적으로 불러오는지 확인
url = 'http://openapi.tago.go.kr/openapi/service/BusSttnInfoInqireService/getSttnNoList?{키}'
result = requests.get(urlparse(url).geturl(), headers={})

xml_obj = xml.text
print(json_obj)
#request.get_method = lambda: 'GET'
#response_body = urlopen(request).read()
#val = []
#json_obj
   # for document in json_obj['documents']:
       # val = [document['address_name'], document['x'], document['y']]

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><gpslati>36.449863</gpslati><gpslong>127.428955</gpslong><nodeid>DJB8002133</nodeid><nodenm>신탄진역</nodenm><nodeno>52790</nodeno></item><item><gpslati>36.449253</gpslati><gpslong>127.42927</gpslong><nodeid>DJB8002134</nodeid><nodenm>신탄진역</nodenm><nodeno>52810</nodeno></item><item><gpslati>36.448536</gpslati><gpslong>127.44346</gpslong><nodeid>DJB8002135</nodeid><nodenm>금강로하스타워</nodenm><nodeno>53000</nodeno></item><item><gpslati>36.4489</gpslati><gpslong>127.44465</gpslong><nodeid>DJB8002136</nodeid><nodenm>금강로하스타워</nodenm><nodeno>52990</nodeno></item><item><gpslati>36.441025</gpslati><gpslong>127.42425</gpslong><nodeid>DJB8002137</nodeid><nodenm>신탄중앙중학교</nodenm><nodeno>52100</nodeno></item><item><gpslati>36.441338</gpslati><gpslong>127.424545</gpslong><nodeid>DJB8002138</nodeid><nodenm>신탄중앙중학교</nodenm><nodeno>52110</nodeno><

In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [104]:
#함수 정리 

def URL(y, x) :
    url = """
http://openapi.tago.go.kr/openapi/service/BusSttnInfoInqireService/getCrdntPrxmtSttnList
?ServiceKey={키}&
gpsLati={}&gpsLong={}""".format(y,x)
    
    # 공백 제거해서 URL로 바꾸어줌
    nospaceURL = re.sub('|\t|\r|\n', '', url)
    return nospaceURL

def PageChange(url, num):
    purl = url + '&pageNo=' + str(num)
    nospaceURL = re.sub('|\t|\r|\n', '', purl)
    return nospaceURL

def PageCount(url):
    result = requests.get(urlparse(url).geturl(), headers={})

    xml_obj = result.text
    soup = BeautifulSoup(xml_obj, 'html.parser')

    pages = soup.select_one('totalcount').get_text()
    return pages
def loopSearch(url):
    list = []
    pages = PageCount(url)
    for i in range(1,int((int(pages)/10))+1):
        url2 = PageChange(url, i)
        result = requests.get(urlparse(url2).geturl(), headers={})

        xml_obj = result.text
        soup = BeautifulSoup(xml_obj, 'html.parser')
        item = soup.select('item')
        for info in item:
    
            list.append(info.select_one('nodenm').get_text())
            list.append(info.select_one('nodeid').get_text())
            list.append(info.select_one('gpslati').get_text())
            list.append(info.select_one('gpslong').get_text())
    
    return list


def makeFrame(list):
    df = pd.DataFrame(columns= ['name', 'ID', 'y', 'x'])
    nm = []
    ID = []
    Y = []
    X = []
    for i in range(0, len(list)):
        
        if i%4 == 0 :
            nm.append(list[i])
        elif i%4 == 1:
            ID.append(list[i])
        elif i%4 == 2:
            Y.append(list[i])
        elif i%4 == 3:
            X.append(list[i])
    df['name'] = nm
    df['ID'] = ID
    df['y'] = Y
    df['x'] = X
    return df

In [147]:
# 1단계 역 근처 정류장들 불러오기


step1 = pd.read_csv('C:/Extracting/raw/step1.csv')
i = step1['x'].count()
count = int(i)
print(step1['x'][2])

for j in range(0, count) :
    url2 = URL(step1['y'][j],step1['x'][j])
    list = loopSearch(url2)     
    bus_stop=makeFrame(list)
    bus_stop.to_csv('C:\Extracting\{}.csv'.format(step1['name'][j]), sep=',',encoding='utf-8-sig')

127.4217747


In [3]:
#도시 코드 구해오는 api
#대전은 25임을 확인

url = 'http://openapi.tago.go.kr/openapi/service/BusSttnInfoInqireService/getCtyCodeList?ServiceKey=키'
result = requests.get(urlparse(url).geturl(), headers={})

xml_obj = result.text
#print(xml_obj)


In [ ]:
"""
정규 표현식으로 태크 추출 
잘 안먹혀서 BeautifulSoup 사용하기로함

#xml 태그에서 <body> 태그 값만 저장 
#body = re.search('<body.*/body>', xml_obj, re.I|re.S)
#print(body)
#xml 태그에서 필요한 <totalCount>
Pages = re.search('<body><response><totalCount>*</totalCount></body></response>', xml_obj, re.I|re.S)


#print(xml_obj)
print(Pages)
"""